In [1]:
import pandas as pd
import json
import time
import os
import dill
from docembedder.bert import BERTEmbedder
from docembedder.analysis import DOCSimilarity
from docembedder.preparation import DOCPreparation
from docembedder.doc2vec import D2VEmbedder

/home/qubix/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preparation

In [2]:
%%time
# # Read and preprocess Patent's documents
# patent_document = DOCPreparation()
# patent_df = patent_document.read_patents()
# patent_df.shape

# # Do some preprocessing here

CPU times: user 0 ns, sys: 2 µs, total: 2 µs
Wall time: 3.81 µs


## Read patent's dataset

In [3]:
%%time
patent_df = pd.read_csv('data/patents_concatenated.csv')
patent_df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/patents_concatenated.csv'

In [4]:
# Removing null values
patent_df.dropna(inplace=True)
patent_df.reset_index(drop=True, inplace=True)
print('shape:', patent_df.shape)

NameError: name 'patent_df' is not defined

#### Dataset overview

In [ ]:
patent_df.index

In [ ]:
patent_df.groupby('year').groups.keys()

In [ ]:
patent_df.groupby(['year']).agg('count')

In [ ]:
# Remove year 0 and year 1953
patent = patent_df[(patent_df['year']!=0) & (patent_df['year']!=1953)]
patent.shape

## Embeddings generation

In [ ]:
# patent_embedder = BERTEmbedder()

# patent_embedder.fit(patent_df['contents'])

## Similarity analysis

In [ ]:
patent_analyser = DOCSimilarity.from_dill()
patent_analyser.compute_similarity()
patent_analyser.df_patents_embeddings


In [ ]:
import pandas as pd
df = pd.read_csv('data/patents_concatenated.csv')
df.shape